In [25]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None  # default='warn'

path = r'REDACTED'
os.chdir(path)
#List of files in folder
file_list = os.listdir(path)

keywords_akkumulerende = ['Akk.', 'Akkumu', 'Akk ', 'Acc'] #Keywords in text for accumulating fund (note Accunia would return True)

def get_institutions(df):
    #Objective: gets the first word in each row of Navn column
    #Returns: list of singular words
    institutions = []
    #Secure Spectrum's funds renamed
    df["Navn"] = df["Navn"].replace(['KF Wealth Invest Secure Market Power ',
                                    'Wealth Invest Secure Globale Aktier',
                                    'Wealth Invest Secure Select Aktier',
                                    'Wealth Invest Secure Globale Obligationer'],
                                    ['Secure Market Power', 
                                     'Secure Globale Aktier', 
                                     'Secure Select Aktier',
                                     'Secure Globale Obligationer'])
    #Othania's funds renamed
    df["Navn"] = df["Navn"].replace(['Wealth Invest AKL Othania Globale Aktier ',
                                    'Investin, Othania Etisk Formuevækst KL',
                                    'KF Wealth Invest AKL Othania Stabil Investering '],
                                    ['Othania Globale Aktier AKL', 
                                     'Othania Etisk Formuevækst KL', 
                                     'Othania Stabil Investering AKL'])
    for value in df.Navn.values:
        if value.split(" ")[0] in ['Investeringsforeningen', 'Kapitalforeningen']:
            institutions.append(value.split(" ")[1])
            continue
        institutions.append(value.split(" ")[0])
    
    return institutions

def iogt_konk(row):
    #Function applicable to DataFrame that returns the I&T fund related to the fund.
    ifr = row['IFR-kategorier']
    if ifr in ["Aktier Norden", "Aktier Danmark"]:
        val = "I&T Nordiske Aktier Large Cap"
    elif ifr == "Kapitalforeninger Aktier":
        val = "I&T Aktier KL"
    elif ifr == "Aktier Globale":
        val = "Globale Bæredygtige Aktier"
    elif ifr == "Obligationer Korte danske":
        val = "I&T Obligationer KL"
    elif ifr == "Obligationer Non-investment Grade":
        val = "I&T Erhvervsobligationer II KL"
    elif ifr in ["Andre alternative investeringsfonde", "Kapitalforeninger Hedgestrategier"]:
        val = "I&T Alternativer KL"
    elif ifr in ["Kapitalforeninger Blandede", "Blandede Balanceret", "Blandede Lav aktieandel", "Blandede Fleksibel", "Blandede Høj aktieandel"]:
        val = "I&T Balance KL"
    else:
        val = ""
    return val

def foreningsfokus(df):
    #Objective: gets type of fund (from keywords in Navn) in each row of Navn column
    #Returns list of singular words
    foreningsfokus = []
    for value in df.Navn.values:
        if 'æredygtig' in value:
            foreningsfokus.append('Bæredygtig')
        elif 'ustain' in value:
            foreningsfokus.append('Bæredygtig')
        elif 'omentum' in value:
            foreningsfokus.append('Momentum')
        elif 'Small ' in value:
            foreningsfokus.append('Small Cap')
        elif 'Large ' in value:
            foreningsfokus.append('Large Cap')
        elif 'alue' in value:
            foreningsfokus.append('Value')
        elif 'dbytte' in value:
            foreningsfokus.append('Udbytte')
        elif 'udb ' in value.lower():
            foreningsfokus.append('Udbytte')
        elif 'Mid ' in value:
            foreningsfokus.append('Mid Cap')
        #If none of the requirements above are met, we append empty string
        else:
            foreningsfokus.append("")
    
    return foreningsfokus

def akkumulerende(df):
    #Objective: Determine whether a fund is accumulating or not based on the name of the fund
    #Returns: list of Ja or Nej
    
    akkumulerende = []
    for value in df.Navn.values:
        lowercase = value.lower()
        if 'akk.' in lowercase:
            akkumulerende.append('Ja')
        elif 'kkumulerende' in lowercase:
            akkumulerende.append('Ja')
        elif 'akk ' in lowercase:
            akkumulerende.append('Ja')
        elif 'acc' in lowercase:
            #Accunia would return True here, so we take them out with if-else
            if 'accunia' in lowercase:
                akkumulerende.append('Nej')
            else:
                akkumulerende.append('Ja')
        else:
            akkumulerende.append('Nej')
    
    return akkumulerende

def marketshare():
    #Objective: Read 'markedsstatistik-MONTH-YEAR' and calculate market shares
    #Returns: Dataframe with Institution, Bruttoformue for latest month and Marketshare
    path3 = r'REDACTED'
    file = os.listdir(path3)[0]
    os.chdir(path3)
    df_formue = pd.read_excel(file,sheet_name='2.1 Formue (A)',skiprows=2,skipfooter=3)
    fonds_navne = list(df_formue['Kursværdi - mio kr.'][:-1])
    column_name = df_formue.columns[7]
    bruttoformue = list(df_formue[column_name][:-1])
    formue_i_alt = list(df_formue[column_name][-1:])[0]

    market_share = []

    for i, formue in enumerate(bruttoformue):
        #Sætter foreningsgrupper der har udenlandske fonde til marketshare = 0.0, da disse ikke indgår i totalen for danske foreninger
        string_navn = fonds_navne[i]
        if string_navn.split('**')[0] != string_navn:
            market_share.append(0.0)
            continue
        #If value is 'nan' (this check works for some reason)
        elif formue != formue:
            market_share.append(0.0)
            continue
        market_share.append(formue/formue_i_alt)
    
    institutions = []
    for navn in fonds_navne:
        if navn.split(" ")[0] in ['Investeringsforeningen', 'Kapitalforeningen']:
            institutions.append(navn.split(" ")[1])
            continue
        institutions.append(navn.split(" ")[0])
    
    #Manually change I&T's name "Investering" to "I&T" to match the Institution column in df
    indexIT = institutions.index('Investering')
    institutions[indexIT] = 'I&T'
    
    return institutions, bruttoformue, market_share

def omkostninger():
    
    path4 = r'REDACTED'
    file = os.listdir(path4)[0]
    os.chdir(path4)
    df_omkostninger = pd.read_excel(file,skiprows=4)
    del df_omkostninger['Unnamed: 1']
    del df_omkostninger['Dato']
    df_omkostninger = df_omkostninger[(df_omkostninger['Navn'] != "Noteret") | (df_omkostninger['Navn'] != "Noteret")]
    
    navne_liste = list(df_omkostninger.Navn.values)
    omkostninger1 = list(df_omkostninger["Resultatafhængig aflønning"].values)
    omkostninger2 = list(df_omkostninger["Indirekte handels.omk."].values)
    omkostninger3 = list(df_omkostninger["Løbende omk. i %"].values)
    åop_list = list(df_omkostninger["ÅOP"].values)
    
    #Convert ÅOP to % and replace , with .
    omkostninger4 = []
    for item in åop_list:
        try:
            omkostninger4.append(float(item.replace(",","."))/100)
        except:
            omkostninger4.append(item)
    
    return navne_liste, omkostninger1, omkostninger2, omkostninger3, omkostninger4 

for file in file_list:
    if len(file_list) != 1:
        print('Make sure the correct file is in the folder - check guide alternatively')
        break
    #We read the Excel file and a specific sheet
    df = pd.read_excel(file, sheet_name = 'Afkast og risikonøgletal', skiprows = 1, skipfooter = 3)
    #Drops missing ISIN values
    df = df[df['ISIN'].notna()]
    #Strips leading spaces from names
    df['Navn'] = df['Navn'].str.lstrip()
    df = df.reset_index()
    del df['index'] #surely we can write this line and the line just above in 1 line?? using inplace = True?
    #Insert new column 'Institution' in column index = 0 using get_institution function
    df.insert(0, 'Institution', get_institutions(df))
    df.insert(4, 'Foreningsfokus', foreningsfokus(df))
    df.insert(5, 'Akkumulerende', akkumulerende(df))
    
    #Marketshare section
    institutions_input, bruttoformue_input, market_share_input = marketshare()
    
    bruttoformue, market_share = [], []

    #Loop over all values in Institution column of df
    for inst in df.Institution.values:
        #If inst is in the df_marketshare Institutions column
        if inst in institutions_input:
            #Set temporary variable index equal to the index so we can retrieve marketshare and bruttoformue
            index = institutions_input.index(inst)
            bruttoformue.append(bruttoformue_input[index])
            market_share.append(market_share_input[index])
            continue
        bruttoformue.append("")
        market_share.append("")
    
    #Divide all rows of specific columns with 100 to convert to percentages
    listofpercentages = ["1 mdr.", "ÅTD", "1 år", "3 år", "5 år", "7 år", "10 år", "std.afv. 3 år", "std.afv. 5 år"]
    for perc in listofpercentages:
        df[perc] = df[perc].div(100)
    
    #Insert Bruttoformue (mio.DKK) in column index = 6
    df.insert(6, 'Bruttoformue', bruttoformue)
    #Insert Market share in column index = 7
    df.insert(7, 'Marketshare', market_share)
    
    #Omkostninger section
    navne_liste, omkostninger1, omkostninger2, omkostninger3, omkostninger4 = omkostninger()
    
    omkostninger1_final, omkostninger2_final, omkostninger3_final, omkostninger4_final = [], [], [], []
    
    #Loop over all values in Navn column of df
    for navn in df.Navn.values:
        #If navn is in the df_omkostninger Navn column
        if navn in navne_liste:
            #Set temporary variable index equal to the index so we can retrieve omkostninger
            index = navne_liste.index(navn)
            omkostninger1_final.append(omkostninger1[index])
            omkostninger2_final.append(omkostninger2[index])
            omkostninger3_final.append(omkostninger3[index])
            omkostninger4_final.append(omkostninger4[index])
            continue
        omkostninger1_final.append(np.nan)
        omkostninger2_final.append(np.nan)
        omkostninger3_final.append(np.nan)
        omkostninger4_final.append(np.nan)
    
    #Add omkostninger columns to final df
    df["Resultatafhængig aflønning"] = omkostninger1_final
    df["Indirekte handels.omk."] = omkostninger2_final
    df["Løbende omk. i %"] = omkostninger3_final
    df["ÅOP"] = omkostninger4_final
    
    #Manually change I&T Alternativer to another IFR Category
    df.loc[df["Navn"].str.contains("I&T Alternativer"), "IFR-kategorier"] = "Andre alternative investeringsfonde"
    #Manually change I&T Aktier and udloddende to another IFR Category
    df.loc[df["Navn"].str.contains("I&T Aktier KL"), "IFR-kategorier"] = "Aktier Norden"
    df.loc[df["Navn"].str.contains("I&T Aktier udloddende KL"), "IFR-kategorier"] = "Aktier Norden"
    #I&T Erhvervsobligationer I skal slettes
    df = df.drop(df[df["Navn"] == "I&T Erhvervsobligationer I KL"].index)
    #I&T NALC skal være bæredygtig
    df.loc[df["Navn"] == "I&T Nordiske Aktier Large Cap", "Foreningsfokus"] = "Bæredygtig"
    
    #Replace Aktier Danmark with Aktier Norden to correctly filter Nordic stocks later on
    df["IFR-kategorier"] = df["IFR-kategorier"].replace(['Aktier Danmark'], 'Aktier Norden')
    #Replace Aktier Kina and Aktier Fjernøsten with Aktier Emerging Markets to correctly filter Emerging Markets later on
    df["IFR-kategorier"] = df["IFR-kategorier"].replace(['Aktier Kina', 'Aktier Fjernøsten', 'Aktier Latinamerika'], 'Aktier Emerging Markets')
    #Replace Hedge funds with Alternatives for filtering later on
    df["IFR-kategorier"] = df["IFR-kategorier"].replace(["Kapitalforeninger Hedgestrategier"], "Andre alternative investeringsfonde")
    
    #Insert column in DataFrame with I&T fund that is in competition with the particular fund
    df["Konkurrent til I&T"] = df.apply(iogt_konk, axis = 1)
    #Reorder DataFrame
    cols = df.columns.tolist()
    cols = cols[0:2] + cols[-1:] + cols[2:-1]
    df = df[cols]
    
    #Manually change I&T Funds competitor reference to themselves rather than others or empty
    df.loc[df["Navn"].str.contains("I&T Erhvervsobligationer II KL"), "Konkurrent til I&T"] = "I&T Kreditobligationer KL"
    df.loc[df["Navn"].str.contains("I&T Obligationer KL"), "Konkurrent til I&T"] = "I&T Obligationer KL"
    df.loc[df["Navn"].str.contains("I&T Aktier KL"), "Konkurrent til I&T"] = "I&T Aktier KL"
    df.loc[df["Navn"].str.contains("I&T Aktier udloddende KL"), "Konkurrent til I&T"] = "I&T Aktier KL"
    
#---------------------------------------------------------------------------
# Konkurrenter: Competing funds with I&T
#---------------------------------------------------------------------------
    
#Liste over konkurrenter fra LLK's overblik
konkurrenter = ["BankInvest", "BankInvest,", "BLS", "Danske", "Fundamental", "Handelsinvest", "Jyske", "Maj", "Nordea", "Nykredit", "Othania", "SEBinvest", "Secure", "C", "Formuepleje", "Stonehenge", "Carnegie", "I&T"]
#Ny DataFrame med konkurrenter
df_konkurrenter = df.loc[df['Institution'].isin(konkurrenter)]

#---------------------------------------------------------------------------
# I&T Fonds konkurrenter: Long table with competitors of I&T
#---------------------------------------------------------------------------

def long_table(df):
    #Returns DataFrame of competitors to I&T funds
    df_konk_nalc = df.loc[df["Konkurrent til I&T"] == "I&T Nordiske Aktier Large Cap"] #Get NALC competition
    #df_konk_nalc = df_konk_nalc[df_konk_nalc['Navn'] != "I&T Aktier KL"] #Remove I&T Aktier from NALC competition
    df_konk_nalc_nalc = df_konk_nalc[df_konk_nalc['Navn'] == "I&T Nordiske Aktier Large Cap"] #Select I&T NALC from NALC competition
    df_konk_nalc = df_konk_nalc[df_konk_nalc['Navn'] != "I&T Nordiske Aktier Large Cap"] #Remove I&T NALC from NALC competition
    df_konk_nalc = pd.concat([df_konk_nalc_nalc, df_konk_nalc]) #Add I&T NALC competition to I&T NALC (so NALC is first)
    
    df_konk_aktier = df.loc[df["Konkurrent til I&T"] == "I&T Aktier KL"] #Get Aktier competition
    df_konk_aktier_aktier = df[df['Navn'] == "I&T Aktier KL"] #Select I&T Aktier from df
    df_konk_aktier_aktierudl = df[df['Navn'] == "I&T Aktier udloddende KL"] #Select I&T Aktier udloddende from df
    df_konk_aktier = df_konk_aktier[df_konk_aktier['Navn'] != "I&T Aktier KL"] #Remove I&T Aktier from NALC competition
    df_konk_aktier = df_konk_aktier[df_konk_aktier['Navn'] != "I&T Aktier udloddende KL"] #Remove I&T Aktier udloddende from NALC competition
    df_konk_aktier = pd.concat([df_konk_aktier_aktier, df_konk_aktier_aktierudl, df_konk_aktier]) #Add I&T Aktier and I&T Aktier udloddende to I&T Aktier competition
    #df_konk_aktier.loc[df_konk_aktier['Navn'] == "I&T Aktier KL", 'Konkurrent til I&T'] = "I&T Aktier KL" #Manually change I&T Aktier KL competitor reference to itself rather than NALC
    #df_konk_aktier.loc[df_konk_aktier['Navn'] == "I&T Aktier udloddende KL", 'Konkurrent til I&T'] = "I&T Aktier udloddende KL"
    
    df_konk_alter = df.loc[df["Konkurrent til I&T"] == "I&T Alternativer KL"]
    df_konk_alter_alter = df_konk_alter[df_konk_alter['Navn'] == "I&T Alternativer KL"] #Select I&T Alternativer from Alternativer competition
    df_konk_alter = df_konk_alter[df_konk_alter['Navn'] != "I&T Alternativer KL"] #Remove I&T Alternativer from Alternativer competition
    df_konk_alter = pd.concat([df_konk_alter_alter, df_konk_alter]) #Add I&T Alternativer competition to I&T Alternativer (so Alternativer is first)
    
    df_konk_bal = df.loc[df["Konkurrent til I&T"] == "I&T Balance KL"]
    df_konk_bal_bal = df_konk_bal[df_konk_bal['Navn'] == "I&T Balance KL"]
    df_konk_bal_alph = df_konk_bal[df_konk_bal['Navn'] == "I&T Alpha KL"]
    df_konk_bal = df_konk_bal[df_konk_bal['Navn'] != "I&T Balance KL"]
    df_konk_bal = pd.concat([df_konk_bal_bal, df_konk_bal_alph, df_konk_bal])
    
    df_konk_obl = df.loc[(df["Konkurrent til I&T"] == "I&T Obligationer KL") & (df["Navn"] != "I&T Obligationer KL")]
    df_konk_obl_obl = df[df['Navn'] == "I&T Obligationer KL"]
    df_konk_obl = pd.concat([df_konk_obl_obl, df_konk_obl])
    df_konk_obl.loc[df_konk_obl['Navn'] == "I&T Obligationer KL", 'Konkurrent til I&T'] = "I&T Obligationer KL"
    
    df_konk_kred = df.loc[df["Konkurrent til I&T"] == "I&T Erhvervsobligationer II KL"]
    df_konk_kred_kred = df[df['Navn'] == "I&T Erhvervsobligationer II KL"]
    df_konk_kred = pd.concat([df_konk_kred_kred, df_konk_kred])
    df_konk_kred.loc[df_konk_kred['Navn'] == "I&T Erhvervsobligationer II KL", 'Konkurrent til I&T'] = "I&T Erhvervsobligationer II KL"
    
    df_konk_gba = df.loc[df["Konkurrent til I&T"] == "Globale Bæredygtige Aktier"] #GBA is not in dataset

    df_konk = pd.concat([df_konk_aktier, 
                         df_konk_nalc, 
                         df_konk_alter, 
                         df_konk_bal, 
                         df_konk_obl, 
                         df_konk_kred, 
                         df_konk_gba])
    df_konk = df_konk.drop(columns = ["ISIN",
                                      "Foreningsfokus", 
                                      "Akkumulerende", 
                                      "Bruttoformue", 
                                      "Marketshare", 
                                      "1 år", 
                                      "7 år", 
                                      "10 år", 
                                      "std.afv. 5 år", 
                                      "Sharpe 3 år", 
                                      "Sharpe 5 år", 
                                      "Indirekte handels.omk.", 
                                      "Løbende omk. i %"])
    return df_konk

df_konk = long_table(df)

#This has been moved up from Industri overblik section per request.
corr = {
    "Europa": "Aktier Europa",
    "USA": "Aktier Nordamerika",
    "Nordiske": "Aktier Norden",
    "Globale": "Aktier Globale",
    "Emerging": "Aktier Emerging Markets",
    "Obligationer": "Obligationer",
    "Blandede": "Blandede",
    "Alternativer": "Andre alternative investeringsfonde"
        }

#Per request from LLK: Manually insert industry overview column to Third Sheet: I&T Fonds konkurrenter
#We use the same approach as clean_table func using corr

def industry_col(row, corr = corr):
    #Function applicable to DataFrame that returns the industry related to the fund.
    ifr = row['IFR-kategorier']
    
    for item in corr:
        if item == corr[item]:
            if corr[item] in ifr:
                return item
        else:
            if ifr == corr[item]:
                return item
    return ""

#Apply industry_col to each df_konk row
df_konk["Industri"] = df_konk.apply(industry_col, axis = 1)
#Drop IFR-kategorier column
df_konk = df_konk.drop(columns = "IFR-kategorier")
#Reorder DataFrame
cols = df_konk.columns.tolist()
cols = cols[0:2] + cols[-1:] + cols[2:-1]
df_konk = df_konk[cols]

#---------------------------------------------------------------------------
# Industri overblik: Clean summarizing tables
#---------------------------------------------------------------------------

def clean_table(df_input, inst, corr):
    #Takes input: DataFrame, Institution and corr (corresponding values of IFR category to type of fund focus in a dict)
    if inst != "Alle":
        df = df_input[df_input["Institution"] == inst]
        try:
            marketshare_inst = df[df['Institution'] == inst]["Marketshare"].mean()
        except:
            marketshare_inst = ""
    else:
        df = df_input
        marketshare_inst = 1
    
    df_clean = pd.DataFrame(columns = ["Markedsandel",
                                          "Europa",
                                          "USA",
                                          "Nordiske", 
                                          "Globale", 
                                          "Emerging",  
                                          "Obligationer", 
                                          "Blandede", 
                                          "Alternativer"], 
                              index = [inst,
                                       "-Antal fonde",
                                       "-Afkast, 1 mdr.",
                                       "-Afkast, ÅTD",
                                       "-Afkast, 3 år",
                                       "-Afkast, 5 år",
                                       "-Std.afv. 3 år",
                                       "-Gns. ÅOP",
                                       "-Resultatafhængige (antal)",
                                       "-Bæredygtige (antal)"]
                                       )
    for item in corr:
        if item == corr[item]:
            length = len(df[df["IFR-kategorier"].str.contains(item)])
            df_clean[item] = ["",
                              [length if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["1 mdr."].mean()*100:.1f}%' if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["ÅTD"].mean()*100:.1f}%' if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["3 år"].mean()*100:.1f}%' if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["5 år"].mean()*100:.1f}%' if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["std.afv. 3 år"].mean()*100:.1f}%' if length > 0 else ""][0],
                              [f'{df[df["IFR-kategorier"].str.contains(item)]["ÅOP"].mean()*100:.2f}%' if length > 0 else ""][0],
                              [len(df[(df["IFR-kategorier"].str.contains(item)) & (df["Resultatafhængig aflønning"] == "Ja")]) if length > 0 else ""][0],#/len(df[df["IFR-kategorier"].str.contains(item)]),
                              [len(df[(df["IFR-kategorier"].str.contains(item)) & (df["Foreningsfokus"] == "Bæredygtig")]) if length > 0 else ""][0]]#/len(df[df["IFR-kategorier"].str.contains(item)])]
        else:
            length = len(df[df["IFR-kategorier"] == corr[item]])
            df_clean[item] = ["", 
                                [length if length > 0 else ""][0], 
                                [f'{df[df["IFR-kategorier"] == corr[item]]["1 mdr."].mean()*100:.1f}%' if length > 0 else ""][0], 
                                [f'{df[df["IFR-kategorier"] == corr[item]]["ÅTD"].mean()*100:.1f}%' if length > 0 else ""][0], 
                                [f'{df[df["IFR-kategorier"] == corr[item]]["3 år"].mean()*100:.1f}%' if length > 0 else ""][0], 
                                [f'{df[df["IFR-kategorier"] == corr[item]]["5 år"].mean()*100:.1f}%' if length > 0 else ""][0], 
                                [f'{df[df["IFR-kategorier"] == corr[item]]["std.afv. 3 år"].mean()*100:.1f}%' if length > 0 else ""][0],
                                [f'{df[df["IFR-kategorier"] == corr[item]]["ÅOP"].mean()*100:.2f}%' if length > 0 else ""][0], 
                                [len(df[(df["IFR-kategorier"] == corr[item]) & (df["Resultatafhængig aflønning"] == "Ja")]) if length > 0 else ""][0], #/len(df[df["IFR-kategorier"] == corr[item]])
                                [len(df[(df["IFR-kategorier"] == corr[item]) & (df["Foreningsfokus"] == "Bæredygtig")]) if length > 0 else ""][0]] #/len(df[df["IFR-kategorier"] == corr[item]])
    df_clean["Markedsandel"] = [[f'{marketshare_inst*100:.2f}%' if marketshare_inst != "" else ""][0],
                                "",
                                "",
                                "",
                                "",
                                "",
                                "",
                                "",
                                "",
                                ""]
    #Replace nan% values with empty values
    for col in df_clean.columns:
        df_clean[col] = df_clean[col].replace('nan%', "")

    return df_clean

df_clean_samlet = clean_table(df, "Alle", corr)
df_clean_konk = clean_table(df_konkurrenter, "Alle", corr)
df_clean_iogt = clean_table(df, "I&T",corr)
df_clean_nykredit = clean_table(df, "Nykredit", corr)
df_clean_danske = clean_table(df, "Danske", corr)
df_clean_nordea = clean_table(df, "Nordea", corr)
df_clean_jyske = clean_table(df, "Jyske", corr)
df_clean_bankinvest = clean_table(df, "BankInvest", corr)
df_clean_formuepleje = clean_table(df, "Formuepleje", corr)
df_clean_maj = clean_table(df, "Maj", corr)
df_clean_handelsinvest = clean_table(df, "Handelsinvest", corr)
df_clean_SEB = clean_table(df, "SEBinvest", corr)
df_clean_othania = clean_table(df, "Othania", corr)
df_clean_secure = clean_table(df, "Secure", corr)
df_clean_bls = clean_table(df, "BLS", corr)
df_clean_c = clean_table(df, "C", corr)
df_clean_carnegie = clean_table(df, "Carnegie", corr)
df_clean_stonehenge = clean_table(df, "Stonehenge", corr)
df_clean_fundamental = clean_table(df, "Fundamental", corr)
df_clean = pd.concat([df_clean_samlet, 
                      df_clean_konk.rename(index={'Alle':'I&T+Konkurrenter'}), #renaming the first index
                      df_clean_iogt, 
                      df_clean_nykredit, 
                      df_clean_danske, 
                      df_clean_nordea, 
                      df_clean_jyske, 
                      df_clean_bankinvest, 
                      df_clean_formuepleje, 
                      df_clean_maj, 
                      df_clean_handelsinvest, 
                      df_clean_SEB, 
                      df_clean_othania, 
                      df_clean_secure, 
                      df_clean_bls, 
                      df_clean_c, 
                      df_clean_carnegie, 
                      df_clean_stonehenge, 
                      df_clean_fundamental])

#---------------------------------------------------------------------------
# Write to Excel
#---------------------------------------------------------------------------

path = r'REDACTED'
os.chdir(path)

filename = 'Masterark til markedsstatistik.xlsx'

with pd.ExcelWriter(filename) as writer:
    df.to_excel(writer, index = False, header = True, sheet_name = "Samlet")
    df_konkurrenter.to_excel(writer, index = False, header = True, sheet_name = "Konkurrenter")
    df_konk.to_excel(writer, index = False, header = True, sheet_name = "I&T Fonds konkurrenter")
    df_clean.to_excel(writer, index = True, header = True, sheet_name = "Industri overblik")
    #df_clean_samlet.to_excel(writer, index = True, header = True, sheet_name = "Table_Samlet")
    #df_clean_iogt.to_excel(writer, index = True, header = True, sheet_name = "Table_I&T")
    #df_clean_nykredit.to_excel(writer, index = True, header = True, sheet_name = "Table_Nykredit")
    #df_clean_danske.to_excel(writer, index = True, header = True, sheet_name = "Table_DanskeBank")
    #df_clean_nordea.to_excel(writer, index = True, header = True, sheet_name = "Table_Nordea")
    #df_clean_jyske.to_excel(writer, index = True, header = True, sheet_name = "Table_Jyske")
    #df_clean_bankinvest.to_excel(writer, index = True, header = True, sheet_name = "Table_BankInvest")
    #df_clean_formuepleje.to_excel(writer, index = True, header = True, sheet_name = "Table_Formuepleje")
    #df_clean_maj.to_excel(writer, index = True, header = True, sheet_name = "Table_Maj")
    #df_clean_handelsinvest.to_excel(writer, index = True, header = True, sheet_name = "Table_Handelsinvest")
    #df_clean_SEB.to_excel(writer, index = True, header = True, sheet_name = "Table_SEBinvest")
    #df_clean_othania.to_excel(writer, index = True, header = True, sheet_name = "Table_Othania")
    #df_clean_secure.to_excel(writer, index = True, header = True, sheet_name = "Table_Secure")
    #df_clean_bls.to_excel(writer, index = True, header = True, sheet_name = "Table_BLS")
    #df_clean_c.to_excel(writer, index = True, header = True, sheet_name = "Table_Cworldwide")
    #df_clean_carnegie.to_excel(writer, index = True, header = True, sheet_name = "Table_Carnegie")
    #df_clean_stonehenge.to_excel(writer, index = True, header = True, sheet_name = "Table_Stonehenge")
    #df_clean_fundamental.to_excel(writer, index = True, header = True, sheet_name = "Table_Fundamental")
print("Finished")

Finished
